# Fine-tuned PaliGemma Inference

환경 설정

In [ ]:
!pip install -q -U transformers
!pip install -q peft
!pip install -q datasets
!pip install -q torch
!pip install -q trl
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from PIL import Image
from sklearn.metrics import accuracy_score
import requests

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
from datasets import load_dataset

from collections import Counter
from tqdm import tqdm

huggingface 로그인을 통한 기 학습 모델 로드 권한 확보

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

모델 로드

In [ ]:
config = PeftConfig.from_pretrained("Donguri-b/paligemma_deepfake_2024")
base_model = PaliGemmaForConditionalGeneration.from_pretrained("google/paligemma-3b-pt-224")
model = PeftModel.from_pretrained(base_model, "Donguri-b/paligemma_deepfake_2024")

In [ ]:
processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

데이터셋 로드

In [ ]:
ds = load_dataset("JamieWithofs/Deepfake-and-real-images")

README.md:   0%|          | 0.00/624 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/424M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/116M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/391M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/140002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10905 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/39428 [00:00<?, ? examples/s]

In [ ]:
Counter(ds['validation']['label'][19100:20100]) # 1000개 데이터 추출 / 라벨 분포 고르도록 확인 후 범위 지정

Counter({0: 541, 1: 459})

### 정확도 측정

In [ ]:
true_labels = []
predicted_labels = []

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
bos_token = processor.tokenizer.bos_token  # BOS 토큰 가져오기

for arg in tqdm(range(len(ds['validation']['label']))[19100:20100]):
    image_x = ds['validation'][arg]['image']
    label_y = ds['validation'][arg]['label']
    true_labels.append(label_y)

    prompt = f"<image> {bos_token} answer Is this image made by AI?"
    image_file = image_x.convert("RGB")

    inputs = processor(prompt, image_file, return_tensors="pt", padding="longest")
    output = model.generate(**inputs, max_new_tokens=20)

    predict_x = processor.decode(output[0], skip_special_tokens=True)[-1:]
    # print(processor.decode(output[0], skip_special_tokens=True)[-1:])
    predicted_labels.append(int(predict_x))

100%|██████████| 1000/1000 [1:04:08<00:00,  3.85s/it]


In [ ]:
# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"정확도: {accuracy * 100:.2f}%")

정확도: 94.80%
